# Investigating Movie Performance

### Team 2: Yashharan Babra, Kevin Brereton, Nicole Caballero, Yesha Patel

The objective of this project is to analyze movie data to answer the following questions:  
  
1. How does the director of a movie affect its rating?  
2. How do the ratings of a movie affect its income?  
3. How do the lead actors of a movie affect its income?  

This will be done using data from IMDB, downloaded from Kaggle in CSV format, as well as web scraping the Metacritic website.

### HTTP code

First, make a function which gets a page from Metacritic's "Movie Releases by Score" list. The input to the function is page number of movies on the "Movie Releases by Score" page. ie. page 0 gets an html page for the top 100 movies, page 1 gets movies 101-200, etc.

Notice that there are some extra headers that are added to the requests.get function. This is because Metacritic will 403 error out anybody who accesses the site without the user-agent http header.

There is also some code added which guarantees that we got back html and didn’t get a 400 error, in case the url of the page has been changed.

In [1]:
from datetime import date, datetime
from typing import Optional, Iterable, Dict, Union
import pandas as pd
from bs4 import BeautifulSoup, element
from time import sleep
import requests
import numpy as np



def getMetaCriticPage(page_num:int)->Optional[str]:
    meta_critic_page_headers={
        "accept": "text/html",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.75 Safari/537.36"
    }
    http_req=requests.get(f"https://www.metacritic.com/browse/movies/score/metascore/all/filtered?sort=desc&page={page_num}",headers=meta_critic_page_headers)
    if http_req.status_code!=200 or "text/html" not in http_req.headers["content-type"]:
#         wait 25 seconds and try again. Metacritic will sometimes reject your request, even if you only request every 5 seconds. to prevent that, try again.
        sleep(25)
        http_req=requests.get(f"https://www.metacritic.com/browse/movies/score/metascore/all/filtered?sort=desc&page={page_num}",headers=meta_critic_page_headers)
        if http_req.status_code!=200 or "text/html" not in http_req.headers["content-type"]:
            return None
    
    return http_req.text

## Web Scraping Code

In this code, we use BeautifulSoup to scrape the data from Metacritic. The entry function of this code is the "getMetaCriticValues" function. In that function, the input is the "Movie Releases by Score" html page, and the output is a generator. That generator yields a dictionary for each movie in the html page. So if the first Metacritic page is the input, the output is a generator, which creates a dictionary for the each movie in the top 100. 

The reason that we used a generator rather than a list was to prevent unnecessary intermediate objects taking up memory (ie. a dict for all 10,000 movies). 

There is also minor text normalization in this code. This is because we wanted to remove all website specific quirks from the data before performing any real data cleaning.

In [2]:
def normalizeTitle(title):
    return title.strip()


def findLastPage(html:str)->int:
    return int(BeautifulSoup(html,'html.parser').find("li",{"class":"page last_page"}).find("a",{"class":"page_num"}).text)
    

def findMetaCriticReleaseDate(movie:BeautifulSoup)->Union[datetime,np.float64]:
    dateText=movie.find("div",{"class":"clamp-details"}).find("span").text
    normalizedDateText=dateText.strip()
    if normalizedDateText =="TBA":
        return np.nan
    return datetime.strptime(normalizedDateText,"%B %d, %Y")


def findMetaCriticScore(movie:BeautifulSoup,score_str:str)->float:
    span=movie.find("span",text=score_str)
    text=str(span.next_sibling.next_sibling.text).strip()
    try:
        return float(text)
    except:
        if text =="tbd":
            return np.nan
        else:
            raise ValueError("unable to find meta critic score, and was not tbd. Most likely the website layout changed")


def getMetaCriticValues(html:str)->Iterable[Dict]:
    soup=BeautifulSoup(html,'html.parser')
    for single_movie in soup.find_all("td",{"class":"clamp-summary-wrap"}):
        release_date=findMetaCriticReleaseDate(single_movie)
        year= release_date.year if type(release_date) == datetime else np.nan
        title=normalizeTitle(str(single_movie.find("h3").text))
        critic_rating=findMetaCriticScore(single_movie,"Metascore:")
        user_rating=findMetaCriticScore(single_movie,"User Score:")
        yield {
            "date_published":release_date,
            "title":title,
            "year":year,
            "metacritic_critic_rating":critic_rating/10,
            "metacritic_user_rating":user_rating
        }



## Main Webscraping Script

Now that those functions were created, we then make the code that web scrapes every page on Metacritic’s "Movie Releases by Score" page. To do that, first we check to see how many "Movie Releases by Score" pages there are on Metacritic. Then we iterate through each "Movie Releases by Score" page and call the getMetaCriticValues function for that page. At the end, we have a dataframe which holds all of the Metacritic data we need.

Note that after each html page it gets, it sleeps for 5 seconds. This is to prevent Metacritic from blocking us.

In [3]:
def iterateThroughMetaCriticValues()->Iterable[Dict]:
    last_num=findLastPage(getMetaCriticPage(0))
    
#     only go to last num, since page count in api starts at 0, but on webpage starts at 1.
    for page_num in range(0,last_num,1):
        page=getMetaCriticPage(page_num)
        if page is not None:
            yield from getMetaCriticValues(page)
            sleep(5)

metacritic=pd.DataFrame(iterateThroughMetaCriticValues(),columns=['date_published','title','year','metacritic_critic_rating','metacritic_user_rating'])
metacritic

,date_published,title,year,metacritic_critic_rating,metacritic_user_rating
0,1941-09-04,Citizen Kane,1941.0,10.0,8.5
1,1972-03-11,The Godfather,1972.0,10.0,9.2
2,1954-09-01,Rear Window,1954.0,10.0,8.8
3,1943-01-23,Casablanca,1943.0,10.0,8.9
4,2014-07-11,Boyhood,2014.0,10.0,7.6
...,...,...,...,...,...
13582,1987-08-22,The Garbage Pail Kids Movie,1987.0,0.1,0.8
13583,2015-06-05,United Passions,2015.0,0.1,0.7
13584,1996-01-12,Bio-Dome,1996.0,0.1,7.2
13585,2005-08-12,Chaos,2005.0,0.1,2.3


In [5]:
# this is temporary since I dont want you guys to need to re-run that long code each time.
# dumps it as a python pickle file
metacritic.to_pickle("metacritic.pickle")

In [36]:
# TEMPORARY: START RUNNING FROM HERE TO AVOID WEB SCRAPING
import pandas as pd
metacritic=pd.read_pickle("metacritic.pickle")
metacritic

,date_published,title,year,metacritic_critic_rating,metacritic_user_rating
0,1941-09-04,Citizen Kane,1941.0,10.0,8.5
1,1972-03-11,The Godfather,1972.0,10.0,9.2
2,1954-09-01,Rear Window,1954.0,10.0,8.8
3,1943-01-23,Casablanca,1943.0,10.0,8.9
4,2014-07-11,Boyhood,2014.0,10.0,7.6
...,...,...,...,...,...
13582,1987-08-22,The Garbage Pail Kids Movie,1987.0,0.1,0.8
13583,2015-06-05,United Passions,2015.0,0.1,0.7
13584,1996-01-12,Bio-Dome,1996.0,0.1,7.2
13585,2005-08-12,Chaos,2005.0,0.1,2.3


## IMDB Data  
IMDB data was obtained by downloading a CSV file from Kaggle (https://www.kaggle.com/stefanoleone992/imdb-extensive-dataset)

In [58]:
import re

def parse_money(string_money):
    # Make sure money is in dollars. Otherwise ignore since it can't be compared with other movies.
    if re.findall('\$', string_money):
        money = re.findall('\d{1,3}', string_money)
        money = int("".join(money))
    else: 
        money = np.nan
    return money

def parse_date(string_date):
    # Check that date is in valid format and if so, convert to datetime object
    if re.search('^\d{4}-\d{2}-\d{2}', string_date):
        date = datetime.strptime(string_date, "%Y-%m-%d")
    else:
        date = np.nan
    return date

def parse_year(string_year):
    # Check that year is in valid format and if so, convert to int
    if re.search('^\d{4}', string_year):
        year = int(string_year)
    else:
        year = np.nan
    return year

def convert_to_list(string_format):
    if string_format == "None":
        return []
    else:
        return string_format.split(",")

# usecols specifies columns we want to add from the csv to our dataframe
# converters specifies how we want to transform string values before adding to the dataframe
imdb = pd.read_csv("IMDB_movies.csv", usecols=['title','year','date_published','genre','duration','country','language','director','actors','avg_vote','votes','budget','usa_gross_income','worldwide_gross_income'], parse_dates=['date_published'], date_parser=parse_date, converters={'year':parse_year, 'genre':convert_to_list, 'duration':int, 'country':convert_to_list, 'language':convert_to_list, 'director':convert_to_list, 'actors':convert_to_list, 'avg_vote':float, 'votes':int, 'budget':parse_money, 'usa_gross_income':parse_money, 'worldwide_gross_income':parse_money})
imdb

,title,year,date_published,genre,duration,country,language,director,actors,avg_vote,votes,budget,usa_gross_income,worldwide_gross_income
0,Miss Jerry,1894.0,1894-10-09,[Romance],45,[USA],[],[Alexander Black],"[Blanche Bayliss, William Courtenay, Chaunce...",5.9,154,NaN,NaN,NaN
1,The Story of the Kelly Gang,1906.0,1906-12-26,"[Biography, Crime, Drama]",70,[Australia],[],[Charles Tait],"[Elizabeth Tait, John Tait, Norman Campbell,...",6.1,589,2250.0,NaN,NaN
2,Den sorte drøm,1911.0,1911-08-19,[Drama],53,"[Germany, Denmark]",[],[Urban Gad],"[Asta Nielsen, Valdemar Psilander, Gunnar He...",5.8,188,NaN,NaN,NaN
3,Cleopatra,1912.0,1912-11-13,"[Drama, History]",100,[USA],[English],[Charles L. Gaskill],"[Helen Gardner, Pearl Sindelar, Miss Fieldin...",5.2,446,45000.0,NaN,NaN
4,L'Inferno,1911.0,1911-03-06,"[Adventure, Drama, Fantasy]",68,[Italy],[Italian],"[Francesco Bertolini, Adolfo Padovan]","[Salvatore Papa, Arturo Pirovano, Giuseppe d...",7.0,2237,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85850,Le lion,2020.0,2020-01-29,[Comedy],95,"[France, Belgium]",[French],[Ludovic Colbeau-Justin],"[Dany Boon, Philippe Katerine, Anne Serra, ...",5.3,398,NaN,NaN,3507171.0
85851,De Beentjes van Sint-Hildegard,2020.0,2020-02-13,"[Comedy, Drama]",103,[Netherlands],"[German, Dutch]",[Johan Nijenhuis],"[Herman Finkers, Johanna ter Steege, Leonie ...",7.7,724,NaN,NaN,7299062.0
85852,Padmavyuhathile Abhimanyu,2019.0,2019-03-08,[Drama],130,[India],[Malayalam],[Vineesh Aaradya],"[Anoop Chandran, Indrans, Sona Nair, Simon ...",7.9,265,NaN,NaN,NaN
85853,Sokagin Çocuklari,2019.0,2019-03-15,"[Drama, Family]",98,[Turkey],[Turkish],[Ahmet Faik Akinci],"[Ahmet Faik Akinci, Belma Mamati, Metin Keçe...",6.4,194,NaN,NaN,2833.0


## Merging IMDB and Metacritic Data  
  
We chose to merge on the combination of movie title and year as there are cases where the same movie title is used in more than one year, such as when remakes of rmovies are made. So merging on the title alone is not enough. We chose to merge on year rather than the release date as there are cases in IMDB data where the year in the release date does not match the release year, and does not match the release date from Metacritic. However, the release years match for the same movies acorss IMDB and Metacritic. For example, Frankenstein in IMDB data has release date of 1935-12-02 but release year of 1931, while release date in Metacritic is 1931-11-21.

In [59]:
# The purpose of retrieving data from Metacritic is to get the ratings. The dataframe only holds rating values and movie information, which will be used to merge with IMDB data. So if any piece of information is missing, those rows can be removed as they are not useful to us. 
metacritic = metacritic.dropna()

# Drop release date from IMDB since there is a duplicate column from Metacritic, which has a more accurate release date
imdb = imdb.drop(columns='date_published')

merged = pd.merge(imdb, metacritic, on=['title', 'year'])
merged

,title,year,genre,duration,country,language,director,actors,avg_vote,votes,budget,usa_gross_income,worldwide_gross_income,date_published,metacritic_critic_rating,metacritic_user_rating
0,Intolerance,1916.0,"[Drama, History]",163,[USA],[English],[D.W. Griffith],"[Lillian Gish, Mae Marsh, Robert Harron, F....",7.8,13875,385907.0,NaN,NaN,1916-09-05,9.9,8.9
1,Frankenstein,1931.0,"[Drama, Horror, Romance]",70,[USA],"[English, Latin]",[James Whale],"[Colin Clive, Mae Clarke, John Boles, Boris...",7.8,63315,291000.0,NaN,1626.0,1931-11-21,9.1,8.5
2,Freaks,1932.0,"[Drama, Horror]",64,[USA],"[English, German, French]",[Tod Browning],"[Wallace Ford, Leila Hyams, Olga Baclanova, ...",7.9,41173,310607.0,NaN,4072.0,1932-02-20,8.0,8.3
3,King Kong,1933.0,"[Adventure, Horror, Sci-Fi]",100,[USA],[English],"[Merian C. Cooper, Ernest B. Schoedsack]","[Fay Wray, Robert Armstrong, Bruce Cabot, F...",7.9,77507,670000.0,NaN,651.0,1933-04-07,9.0,8.4
4,Fantasia,1940.0,"[Animation, Family, Fantasy]",125,[USA],[English],"[James Algar, Samuel Armstrong]","[Deems Taylor, Leopold Stokowski, The Philad...",7.8,86795,2280000.0,76408097.0,76411401.0,1940-11-13,9.6,8.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2493,Palm Springs,2020.0,"[Comedy, Fantasy, Mystery]",90,"[USA, Hong Kong]",[English],[Max Barbakow],"[Andy Samberg, Cristin Milioti, J.K. Simmons...",7.4,44712,NaN,164000.0,338427.0,2020-07-10,8.3,7.9
2494,Mickey and the Bear,2019.0,[Drama],88,[USA],[English],[Annabelle Attanasio],"[Camila Morrone, James Badge Dale, Calvin De...",6.6,873,NaN,57188.0,57188.0,2019-11-13,7.9,5.8
2495,American Son,2019.0,[Drama],90,[USA],[English],[Kenny Leon],"[Kerry Washington, Steven Pasquale, Jeremy J...",5.7,5917,NaN,NaN,NaN,2019-11-01,3.4,6.2
2496,The Kissing Booth 2,2020.0,"[Comedy, Romance]",134,"[UK, USA]",[English],[Vince Marcello],"[Joey King, Joel Courtney, Jacob Elordi, Mo...",5.9,15189,NaN,NaN,NaN,2020-07-24,3.9,3.3


## How does the director of a movie affect its rating?

In [63]:
# Ensure that there are no missing values for directors or IMDB ratings. We don't need to check that Metacritic ratings are missing as we only merged rows from the metacritic dataframe without missing values.
director_rating_df = merged.dropna(subset=['director', 'avg_vote'])
director_rating_df

# YOU CAN CONTINUE WITH ANY OTHER PROCESSING NEEDED TO ANSWER THIS QUESTION

,title,year,genre,duration,country,language,director,actors,avg_vote,votes,budget,usa_gross_income,worldwide_gross_income,date_published,metacritic_critic_rating,metacritic_user_rating
0,Intolerance,1916.0,"[Drama, History]",163,[USA],[English],[D.W. Griffith],"[Lillian Gish, Mae Marsh, Robert Harron, F....",7.8,13875,385907.0,NaN,NaN,1916-09-05,9.9,8.9
1,Frankenstein,1931.0,"[Drama, Horror, Romance]",70,[USA],"[English, Latin]",[James Whale],"[Colin Clive, Mae Clarke, John Boles, Boris...",7.8,63315,291000.0,NaN,1626.0,1931-11-21,9.1,8.5
2,Freaks,1932.0,"[Drama, Horror]",64,[USA],"[English, German, French]",[Tod Browning],"[Wallace Ford, Leila Hyams, Olga Baclanova, ...",7.9,41173,310607.0,NaN,4072.0,1932-02-20,8.0,8.3
3,King Kong,1933.0,"[Adventure, Horror, Sci-Fi]",100,[USA],[English],"[Merian C. Cooper, Ernest B. Schoedsack]","[Fay Wray, Robert Armstrong, Bruce Cabot, F...",7.9,77507,670000.0,NaN,651.0,1933-04-07,9.0,8.4
4,Fantasia,1940.0,"[Animation, Family, Fantasy]",125,[USA],[English],"[James Algar, Samuel Armstrong]","[Deems Taylor, Leopold Stokowski, The Philad...",7.8,86795,2280000.0,76408097.0,76411401.0,1940-11-13,9.6,8.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2493,Palm Springs,2020.0,"[Comedy, Fantasy, Mystery]",90,"[USA, Hong Kong]",[English],[Max Barbakow],"[Andy Samberg, Cristin Milioti, J.K. Simmons...",7.4,44712,NaN,164000.0,338427.0,2020-07-10,8.3,7.9
2494,Mickey and the Bear,2019.0,[Drama],88,[USA],[English],[Annabelle Attanasio],"[Camila Morrone, James Badge Dale, Calvin De...",6.6,873,NaN,57188.0,57188.0,2019-11-13,7.9,5.8
2495,American Son,2019.0,[Drama],90,[USA],[English],[Kenny Leon],"[Kerry Washington, Steven Pasquale, Jeremy J...",5.7,5917,NaN,NaN,NaN,2019-11-01,3.4,6.2
2496,The Kissing Booth 2,2020.0,"[Comedy, Romance]",134,"[UK, USA]",[English],[Vince Marcello],"[Joey King, Joel Courtney, Jacob Elordi, Mo...",5.9,15189,NaN,NaN,NaN,2020-07-24,3.9,3.3


## How do the ratings of a movie affect its income?

In [64]:
# Ensure that there are no missing values for worldwide gross income and IMDB ratings. We don't need to check that Metacritic ratings are missing as we only merged rows from the metacritic dataframe without missing values.
rating_income_df = merged.dropna(subset=['worldwide_gross_income', 'avg_vote'])
rating_income_df

# YOU CAN CONTINUE WITH ANY OTHER PROCESSING NEEDED TO ANSWER THIS QUESTION
# If we also want to examine usa income, add usa_gross_income to the subset above

,title,year,genre,duration,country,language,director,actors,avg_vote,votes,budget,usa_gross_income,worldwide_gross_income,date_published,metacritic_critic_rating,metacritic_user_rating
1,Frankenstein,1931.0,"[Drama, Horror, Romance]",70,[USA],"[English, Latin]",[James Whale],"[Colin Clive, Mae Clarke, John Boles, Boris...",7.8,63315,291000.0,NaN,1626.0,1931-11-21,9.1,8.5
2,Freaks,1932.0,"[Drama, Horror]",64,[USA],"[English, German, French]",[Tod Browning],"[Wallace Ford, Leila Hyams, Olga Baclanova, ...",7.9,41173,310607.0,NaN,4072.0,1932-02-20,8.0,8.3
3,King Kong,1933.0,"[Adventure, Horror, Sci-Fi]",100,[USA],[English],"[Merian C. Cooper, Ernest B. Schoedsack]","[Fay Wray, Robert Armstrong, Bruce Cabot, F...",7.9,77507,670000.0,NaN,651.0,1933-04-07,9.0,8.4
4,Fantasia,1940.0,"[Animation, Family, Fantasy]",125,[USA],[English],"[James Algar, Samuel Armstrong]","[Deems Taylor, Leopold Stokowski, The Philad...",7.8,86795,2280000.0,76408097.0,76411401.0,1940-11-13,9.6,8.5
5,Pinocchio,1940.0,"[Animation, Comedy, Family]",88,[USA],[English],"[Norman Ferguson, T. Hee]","[Mel Blanc, Don Brodie, Stuart Buchanan, Wa...",7.4,127618,2600000.0,84254167.0,121892045.0,1940-02-23,9.9,8.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2488,Jexi,2019.0,"[Comedy, Romance]",84,"[USA, Canada]",[English],"[Jon Lucas, Scott Moore]","[Adam Devine, Alexandra Shipp, Rose Byrne, ...",6.1,17038,5000000.0,6546159.0,9341824.0,2019-10-11,3.9,4.0
2489,The Death of Dick Long,2019.0,"[Comedy, Crime, Drama]",100,[USA],[English],[Daniel Scheinert],"[Michael Abbott Jr., Virginia Newcomb, Andre...",6.3,3504,NaN,36856.0,36856.0,2019-09-27,6.9,5.4
2490,The Roads Not Taken,2020.0,[Drama],85,"[UK, Sweden, USA, Poland, Spain]",[English],[Sally Potter],"[Javier Bardem, Elle Fanning, Branka Katic, ...",4.9,753,NaN,3518.0,3518.0,2020-03-13,4.2,3.0
2493,Palm Springs,2020.0,"[Comedy, Fantasy, Mystery]",90,"[USA, Hong Kong]",[English],[Max Barbakow],"[Andy Samberg, Cristin Milioti, J.K. Simmons...",7.4,44712,NaN,164000.0,338427.0,2020-07-10,8.3,7.9


## How do the lead actors of a movie affect its income?

In [65]:
# Ensure that there are no missing values for worldwide gross income and actors
actor_income_df = merged.dropna(subset=['worldwide_gross_income', 'actors'])
actor_income_df

# YOU CAN CONTINUE WITH ANY OTHER PROCESSING NEEDED TO ANSWER THIS QUESTION
# If we also want to examine usa income, add usa_gross_income to the subset above

,title,year,genre,duration,country,language,director,actors,avg_vote,votes,budget,usa_gross_income,worldwide_gross_income,date_published,metacritic_critic_rating,metacritic_user_rating
1,Frankenstein,1931.0,"[Drama, Horror, Romance]",70,[USA],"[English, Latin]",[James Whale],"[Colin Clive, Mae Clarke, John Boles, Boris...",7.8,63315,291000.0,NaN,1626.0,1931-11-21,9.1,8.5
2,Freaks,1932.0,"[Drama, Horror]",64,[USA],"[English, German, French]",[Tod Browning],"[Wallace Ford, Leila Hyams, Olga Baclanova, ...",7.9,41173,310607.0,NaN,4072.0,1932-02-20,8.0,8.3
3,King Kong,1933.0,"[Adventure, Horror, Sci-Fi]",100,[USA],[English],"[Merian C. Cooper, Ernest B. Schoedsack]","[Fay Wray, Robert Armstrong, Bruce Cabot, F...",7.9,77507,670000.0,NaN,651.0,1933-04-07,9.0,8.4
4,Fantasia,1940.0,"[Animation, Family, Fantasy]",125,[USA],[English],"[James Algar, Samuel Armstrong]","[Deems Taylor, Leopold Stokowski, The Philad...",7.8,86795,2280000.0,76408097.0,76411401.0,1940-11-13,9.6,8.5
5,Pinocchio,1940.0,"[Animation, Comedy, Family]",88,[USA],[English],"[Norman Ferguson, T. Hee]","[Mel Blanc, Don Brodie, Stuart Buchanan, Wa...",7.4,127618,2600000.0,84254167.0,121892045.0,1940-02-23,9.9,8.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2488,Jexi,2019.0,"[Comedy, Romance]",84,"[USA, Canada]",[English],"[Jon Lucas, Scott Moore]","[Adam Devine, Alexandra Shipp, Rose Byrne, ...",6.1,17038,5000000.0,6546159.0,9341824.0,2019-10-11,3.9,4.0
2489,The Death of Dick Long,2019.0,"[Comedy, Crime, Drama]",100,[USA],[English],[Daniel Scheinert],"[Michael Abbott Jr., Virginia Newcomb, Andre...",6.3,3504,NaN,36856.0,36856.0,2019-09-27,6.9,5.4
2490,The Roads Not Taken,2020.0,[Drama],85,"[UK, Sweden, USA, Poland, Spain]",[English],[Sally Potter],"[Javier Bardem, Elle Fanning, Branka Katic, ...",4.9,753,NaN,3518.0,3518.0,2020-03-13,4.2,3.0
2493,Palm Springs,2020.0,"[Comedy, Fantasy, Mystery]",90,"[USA, Hong Kong]",[English],[Max Barbakow],"[Andy Samberg, Cristin Milioti, J.K. Simmons...",7.4,44712,NaN,164000.0,338427.0,2020-07-10,8.3,7.9
